# Discovering Sentinel-1 candidate pairs for InSAR

In [ ]:
%load_ext autoreload
%autoreload 2

import json

import geopandas as gpd
import folium
import folium.plugins
import logging
from json import dumps

from eodag import EODataAccessGateway
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

## Search

In [ ]:
# load a geometry
file_aoi = "/data/aoi_Maroc.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
# testing data search
search_criteria = {
    # "productType": "S2_MSI_L1C",
    "productType": "S1_SAR_SLC",
    "start": "2021-03-01",
    "end": "2021-03-30",
    "geom": shp
}
results, _ = dag.search(**search_criteria)

In [ ]:
print(gdf.sample().columns)

In [ ]:
# Convert results to geodataframe
gj = results.as_geojson_object()
gdf = gpd.read_file(dumps(gj))

In [ ]:
print(gdf.iloc[1].keys())

## Visualize by groups

Let's group the products by
- Relative orbit
- Overlapping

In [ ]:
# check for geometrical overlap
def has_overlap(geom1, geom2, tolerance=0.01):
    intersection = geom1.intersection(geom2)
    return intersection.area / min(geom1.area, geom2.area) >= (1 - tolerance)

# make groups of almost fully overlapping products
def group_by_overlap(dfg):
    init_list = list(dfg.index)
    groups = []
    while len(init_list) > 0:
        geom_a = dfg.geometry.loc[init_list[0]]
        grp = []
        grp.append(init_list.pop(0))
        i = 0
        while i < len(init_list):
            geom_b = dfg.geometry.loc[init_list[i]]
            if has_overlap(geom_a, geom_b):
                grp.append(init_list.pop(i))
            else:
                i += 1
        groups.append(grp)
    return groups


In [ ]:
ll = gdf.total_bounds.reshape(2, 2)
# folium inverts coordinates
bbox = list(map(lambda x: [x[1], x[0]], ll))

m = folium.Map()
by_orbit = gdf.groupby('relativeOrbitNumber')
group_indices = []
for idx in by_orbit.groups:
    dfg = by_orbit.get_group(idx).explode(index_parts=False)

    # group overlapping
    group_indices = group_by_overlap(dfg)    
    for g in group_indices:
        sel = gdf.loc[g]
        folium.GeoJson(sel.unary_union).add_to(m)
    
    print(group_indices)
m.fit_bounds(bbox)
m

In [ ]:
ll = gdf.total_bounds.reshape(2, 2)
# folium inverts coordinates
bbox = list(map(lambda x: [x[1], x[0]], ll))
m = folium.Map()
folium.GeoJson(sel.unary_union).add_to(m)
# folium.GeoJson(sel.astype(str).to_json()).add_to(m)
m.fit_bounds(bbox)
m

In [ ]:
import shapely


In [ ]:
folium.Polygon?